In [ ]:
import numpy as np
import cv2
import sys
import pandas as pd


def getAngleBetween(a, b):
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    cosTheta =  dot_product / (norm_a * norm_b)
    return (np.arccos(cosTheta) * 180) / 3.1415


    
def LMedS(lines, n = 100):
    
    leverVectors = []
    for lineIndex in range(lines.shape[0]):
        leverVectors.append(getLeverVector(lines[lineIndex]))
        
    medianResiduals = []
    minResidual = sys.float_info.max
    minI = None, minJ = None, minM = None, minV = None
    
    for m in range(0, n):
        
        lineIndices = np.random.randint(low = 0, high = lines.shape[0], size = 2).tolist()
        i = lineIndices[0]
        j = lineIndices[1]
        n1 = leverVectors[i]
        n2 = leverVectors[j]
        v = getVanishingDirection(n1, n2)
        residuals = []
        for k in range(lines.shape[0]):
            if k != i and k != j:
                nk = leverVectors[k]
                r = getResidual(nk, v)
                residuals.append( r )
                
        
        medResidual = np.nanmedian(np.array(residuals))
        medianResiduals.append(medResidual)
        
        if medResidual < minResidual:
            minResidual = medResidual
            minM = m
            minI = i
            minJ = j
            minV = v
            
    
    print("residual: " + str(minResidual))
    return minV, minResidual


def getPixelCorrdinatesOfVP(v):
    xH = np.dot(fixedMtx, v)
    xH = xH / xH[2]
    return (xH[0], xH[1])
    
def getCameraPoint(pixelPoint):
    return np.dot(KInverse,np.array([pixelPoint[0], pixelPoint[1], 1]))

def getLeverVector(points):
    pc1 = getCameraPoint(points[0])
    pc2 = getCameraPoint(points[1])
    n = np.cross(pc1, pc2)
    return n / np.linalg.norm(n) 


def getVanishingDirection(n1, n2):
    v = np.cross(n1, n2)
    return v / np.linalg.norm(v)
    

def getResidual(n, v):
    r = np.linalg.norm(np.dot(n, v))
    return r

   

In [ ]:
fixedMtx = np.array([[967.74176025,   0.0,         599.80400823],
                     [  0.0,         979.05285645, 465.81112285],
                     [  0.0,           0.0,           1.0   ]])
KInverse = np.linalg.inv(fixedMtx)
df = pd.read_excel(io='horizontalLines.xlsx')
horizontalLines = np.load(df)
df = pd.read_excel(io='verticalLines.xlsx')
verticalLines = np.load('verticalLines.xlsx')

In [ ]:
horVP, horResidual, _, _ = LMedS(horizontalLines)
vertVP, vertResidual, _, _ = LMedS(verticalLines)    

In [ ]:
getAngleBetween(horVP, vertVP)

In [ ]:
transversalVP = np.cross(horVP, vertVP)
transversalVP = transversalVP / np.linalg.norm(transversalVP)

In [ ]:
print(horVP)
print(vertVP)
print(transversalVP)
print(getPixelCorrdinatesOfVP(horVP))
print(getPixelCorrdinatesOfVP(vertVP))